In [113]:
# General utilities. 
import sys, os, re, ast, time, json
import msgpack, attrs, math, csv
import numpy as np
import pandas as pd
import subprocess as sp
import matplotlib.pyplot as plt
from natsort import natsorted
from itertools import compress
from pypdf import PdfReader
from pprint import pprint
from pathlib import Path

import torch
from lightning import pytorch as pl
from lightning.pytorch import __version__
from lightning.pytorch.utilities.parsing import AttributeDict

from chemprop import data, featurizers, models, nn
from chemprop.nn import metrics
from chemprop.models import multi
from chemprop.models.model import MPNN
from chemprop.utils import Factory
from chemprop.utils.v1_to_v2 import convert_model_dict_v1_to_v2
from chemprop.nn.agg import AggregationRegistry
from chemprop.nn.message_passing import AtomMessagePassing, BondMessagePassing
from chemprop.nn.metrics import LossFunctionRegistry, MetricRegistry
from chemprop.nn.predictors import PredictorRegistry
from chemprop.nn.transforms import UnscaleTransform

# "Big three" chemical packages: rdkit, openbabel, and molli.  
import rdkit as rd
from rdkit import Chem, RDConfig
from rdkit.Chem import Draw, AllChem, rdBase, rdMolAlign, rdMolDescriptors, rdDetermineBonds
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Geometry import Point3D
from openbabel import openbabel as ob
m2_m2=ob.OBConversion(); m2_m2.SetInAndOutFormats("mol2", "mol2")
smi_m2=ob.OBConversion(); smi_m2.SetInAndOutFormats("smi", "mol2")
xyz_m2=ob.OBConversion(); xyz_m2.SetInAndOutFormats("xyz", "mol2")
import molli as ml
import molli.visual
from molli.external import _rdkit as mrd
from molli.external import openbabel as mob

# Other chemical packages + py3Dmol setup. 
import cclib, cirpy, py3Dmol
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'
py3Dview={'stick':{'colorscheme':'grayCarbon', "radius":0.1}, 'sphere': {'colorscheme':'grayCarbon', "scale": 0.15}}
py3Dviewstyle={'style':'outline','color':'black','width':0.01, "data-backgroundalpha":0}

True

True

True

In [11]:
def viz2D(mol: Chem.rdchem.Mol):
    plt.axis('off')
    plt.imshow(Draw.MolToImage(mol))
    plt.show()

In [14]:
def viz3D(mol: ml.Molecule):
    conv_mol=m2conv.WriteString(mob.to_obmol(mol))
    v=py3Dmol.view(data=conv_mol, style=py3Dview)
    return v.setViewStyle(py3Dviewstyle) # or v.show()

In [99]:
# CSV should be in same directory, i.e. coefficiency/Deep4Chem_chemprop
csv_name = "d4c_ext_coef_all_data.csv" 
d4c_df = pd.read_csv(csv_name)
d4c_df = d4c_df[d4c_df['Solvent'] != 'gas'] # these failed anyways as per verbose.log; 8018 molecules -- checked.
d4c_df.head()
d4c_df.to_csv('d4c_extCoef_degassed.csv') # this is the csv to actually use!

,Chromophore,Solvent,Absorption max (nm),log(e/mol-1 dm3 cm-1)
0,c1ccc2ccccc2c1,C1CCCCC1,286.0,3.58
1,C[Si](C)(C)c1cccc2ccccc12,C1CCCCC1,294.0,3.73
2,C[SiH](C)c1cccc2ccccc12,C1CCCCC1,294.0,3.75
3,CCCC[Si](C)(C)c1cccc2ccccc12,C1CCCCC1,294.0,3.74
4,CC(C)(C)[Si](C)(C)c1cccc2ccccc12,C1CCCCC1,295.0,3.78


In [117]:
chemprop_dir = Path.cwd()
chemprop_dir

PosixPath('/Users/davidpolefrone/coefficiency/Deep4Chem_chemprop')

In [148]:
model_v1_input_path =  chemprop_dir / "fold_0/model_0/model.pt" # path to v1 model .pt file
model_v2_output_path = chemprop_dir / "fold_0/converted_model_0.ckpt" # path to save the converted model .ckpt file

In [166]:
model_v1_dict = torch.load(model_v1_input_path, weights_only=False, map_location=torch.device('cpu'))

In [144]:
model_v1_dict = torch.load(model_v1_input_path, weights_only=False, map_location=torch.device('cpu'))
model_v1_dict['args'].__dict__['loss_function'] = 'mse'
model_v1_dict['state_dict']['readout.1.weight'] = model_v1_dict['state_dict']['ffn.1.weight']
model_v1_dict['state_dict']['readout.1.bias'] = model_v1_dict['state_dict']['ffn.1.bias']
model_v1_dict['state_dict']['readout.4.weight'] = model_v1_dict['state_dict']['ffn.4.weight']
model_v1_dict['state_dict']['readout.4.bias'] = model_v1_dict['state_dict']['ffn.4.bias']
model_v1_dict['state_dict']['readout.7.weight'] = model_v1_dict['state_dict']['ffn.7.weight']
model_v1_dict['state_dict']['readout.7.bias'] = model_v1_dict['state_dict']['ffn.7.bias']

In [153]:
model_v2_dict = convert_model_dict_v1_to_v2(model_v1_dict)

In [147]:
torch.save(model_v2_dict, model_v2_output_path)

In [174]:
d4c_df.columns._get_indexer_strict(['Chromophore', 'Solvent'], "columns")

(Index(['Chromophore', 'Solvent'], dtype='object'), array([0, 1]))

In [167]:
d4c_df['Chromophore']

0                                          c1ccc2ccccc2c1
1                               C[Si](C)(C)c1cccc2ccccc12
2                                 C[SiH](C)c1cccc2ccccc12
3                            CCCC[Si](C)(C)c1cccc2ccccc12
4                        CC(C)(C)[Si](C)(C)c1cccc2ccccc12
                              ...                        
8027             c1cc2c3ccc[n+]4cccc(c5ccc[n+](c1)c25)c34
8028             c1cc2c3ccc[n+]4cccc(c5ccc[n+](c1)c25)c34
8029             c1cc2c3ccc[n+]4cccc(c5ccc[n+](c1)c25)c34
8030      c1cc2c3c(c1)-c1cccc4cccc(c14)B3c1cccc3cccc-2c13
8031    c1cc2c3c(c1)-c1ccc4c5c(ccc(c15)B3c1ccc3c5c(ccc...
Name: Chromophore, Length: 8018, dtype: object

In [98]:
# non-default hyperparameters:
# --batch-size 50 --message-hidden-dim 1900 --depth 5 --dropout 0.1 --epochs 200

# --data_path becomes --data-path (-i)
# --dataset_type becomes --task-type (-t)
# --num_folds becomes --num-folds (-k)
# --config_path becomes --config-path
# --epochs constant
# --number_of_molecules is removed! Need to supply multiple inputs to --smiles-columns (-s). 
# --save_dir becomes --save-dir/--output-dir (-o). 

# Use --verbose (-v) for non-default verbose logging. 

### See chemprop/examples/training_regression_multicomponent.ipynb.

smiles_columns = ['Chromophore', 'Solvent']
target_columns = ['Absorption max (nm)', 'log(e/mol-1 dm3 cm-1)']

In [36]:
args = 'args.json'
with open('args.json', 'r') as file:
    args = json.load(file)
v1_cli_entry = args['reproducibility']['command_line']
pprint(args)

{'activation': 'ReLU',
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_descriptor_scaling': True,
 'atom_descriptors': None,
 'atom_descriptors_path': None,
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'batch_size': 50,
 'bias': False,
 'bond_feature_scaling': True,
 'bond_features_path': None,
 'bond_features_size': 0,
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,
 'class_balance': False,
 'config_path': 'hyperopt_all_data_2d_feats.json',
 'crossval_index_dir': None,
 'crossval_index_file': None,
 'crossval_index_sets': None,
 'cuda': True,
 'data_path': 'd4c_ext_coef_all_data.csv',
 'data_weights_path': None,
 'dataset_type': 'regression',
 'depth': 5,
 'device': {'_type': 'python_object (type = device)',
            '_value': 'gASVHwAAAAAAAACMBXRvcmNolIwGZGV2aWNllJOUjARjdWRhlIWUUpQu'},
 'dropout': 0.1,
 'empty_cache': False,
 'ensemble_size': 1,
 'epochs': 2

In [58]:
smiss = d4c_df.loc[:, smiles_columns].values
ys = d4c_df.loc[:, target_columns].values

all_data = [[data.MoleculeDatapoint.from_smi(smis[0], y) for smis, y in zip(smiss, ys)]]
all_data += [[data.MoleculeDatapoint.from_smi(smis[i]) for smis in smiss] for i in range(1, len(smiles_columns))]

In [59]:
component_to_split_by = 0 # index of the component to use for structure based splits
mols = [d.mol for d in all_data[component_to_split_by]]
train_indices, val_indices, test_indices = data.make_split_indices(mols, "random", (0.8, 0.1, 0.1))
train_data, val_data, test_data = data.split_data_by_indices(
    all_data, train_indices, val_indices, test_indices
)

In [91]:
#featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

train_datasets = [data.MoleculeDataset(train_data[0][i]) for i in range(len(smiles_columns))]
val_datasets = [data.MoleculeDataset(val_data[0][i]) for i in range(len(smiles_columns))]
test_datasets = [data.MoleculeDataset(test_data[0][i]) for i in range(len(smiles_columns))]

In [92]:
train_mcdset = data.MulticomponentDataset(train_datasets)
scaler = train_mcdset.normalize_targets()
val_mcdset = data.MulticomponentDataset(val_datasets)
val_mcdset.normalize_targets(scaler)
test_mcdset = data.MulticomponentDataset(test_datasets)

StandardScaler()

In [93]:
train_loader = data.build_dataloader(train_mcdset)
val_loader = data.build_dataloader(val_mcdset, shuffle=False)
test_loader = data.build_dataloader(test_mcdset, shuffle=False)

In [110]:
mpnn = nn.BondMessagePassing(d_h=1900, depth=5, dropout=0.1)

In [111]:
agg = nn.MeanAggregation()
output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
ffn = nn.RegressionFFN(
    input_dim=mpnn.output_dim,
    output_transform=output_transform,
)

In [112]:
metric_list = [metrics.RMSE()] # Only the first metric is used for training and early stopping

In [77]:
def convert_v1_args_to_v2_hyper_parameters(model_v1_dict: dict) -> dict:
    """Converts v1 model dictionary to v2 hyper_parameters dictionary"""
    hyper_parameters_v2 = {}

    args_v1 = model_v1_dict # altered to allow for passing in just args. 
    hyper_parameters_v2["batch_norm"] = False
    hyper_parameters_v2["metrics"] = [Factory.build(MetricRegistry[args_v1['metric']])] # 'rmse'
    hyper_parameters_v2["warmup_epochs"] = args_v1['warmup_epochs'] # 2.0
    hyper_parameters_v2["init_lr"] = args_v1['init_lr'] # 0.0001
    hyper_parameters_v2["max_lr"] = args_v1['max_lr'] # 0.001
    hyper_parameters_v2["final_lr"] = args_v1['final_lr'] # 0.0001

    # convert the message passing block -- don't have a state_dict here.
    #W_i_shape = model_v1_dict["state_dict"]["encoder.encoder.0.W_i.weight"].shape
    #W_h_shape = model_v1_dict["state_dict"]["encoder.encoder.0.W_h.weight"].shape
    #W_o_shape = model_v1_dict["state_dict"]["encoder.encoder.0.W_o.weight"].shape
    # hardcoded from verbose.log. 
    W_i_shape = [147, 1900]
    W_h_shape = [1900, 1900]
    W_o_shape = [2033, 1900]

    d_h = W_i_shape[0]
    d_v = W_o_shape[1] - d_h
    d_e = W_h_shape[1] - d_h if args_v1['atom_messages'] else W_i_shape[1] - d_v

    hyper_parameters_v2["message_passing"] = AttributeDict(
        {
            "activation": args_v1['activation'], # 'ReLU'
            "bias": args_v1['bias'], # False
            "cls": BondMessagePassing if not args_v1['atom_messages'] else AtomMessagePassing, # False; BondMP
            "d_e": d_e,  # the feature dimension of the edges
            "d_h": args_v1['hidden_size'],  # dimension of the hidden layer
            "d_v": d_v,  # the feature dimension of the vertices
            "d_vd": None,  # ``d_vd`` is the number of additional features that will be concatenated to atom-level features *after* message passing
            "depth": args_v1['depth'], # 5
            "dropout": args_v1['dropout'], # 0.1
            "undirected": args_v1['undirected'], # False
        }
    )

    # convert the aggregation block
    hyper_parameters_v2["agg"] = {
        "dim": 0,  # in v1, the aggregation is always done on the atom features
        "cls": AggregationRegistry[args_v1['aggregation']],
    }
    if args_v1['aggregation'] == "norm":
        hyper_parameters_v2["agg"]["norm"] = args_v1['aggregation_norm'] # mean

    # convert the predictor block
    if args_v1['target_weights'] is not None:
        task_weights = torch.tensor(args_v1['target_weights']).unsqueeze(0)
    else:
        task_weights = torch.ones(args_v1['num_tasks']).unsqueeze(0) # 2 tasks. 

    hyper_parameters_v2["predictor"] = AttributeDict(
        {
            "activation": args_v1['activation'], # 'ReLU'
            "cls": PredictorRegistry[args_v1['dataset_type']], # 'regression'
            "criterion": Factory.build(
                LossFunctionRegistry['mse'], task_weights=task_weights # hardcoded to default from v1
            ), 
            "task_weights": None,
            "dropout": args_v1['dropout'], # 0.1
            "hidden_dim": args_v1['ffn_hidden_size'], # 1900
            "input_dim": args_v1['hidden_size'], # 1900
            "n_layers": args_v1['ffn_num_layers'] - 1, # 3 - 1
            "n_tasks": args_v1['num_tasks'], # 2
        }
    )
    '''
    if args_v1.dataset_type == "regression":
        hyper_parameters_v2["predictor"]["output_transform"] = UnscaleTransform(
            model_v1_dict["data_scaler"]["means"], model_v1_dict["data_scaler"]["stds"]
        )
    '''

    return hyper_parameters_v2